In [ ]:
# Copyright 2022 Huawei Technologies Co., Ltd.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# author: Georg Anegg

In [ ]:
filename = 'path/to/file'
num_entries = 250 # number of (hyperedge_index, node_index) entries to read

In [ ]:
from collections import defaultdict 
import networkx as nx
import matplotlib.pyplot as plt
import hypernetx as hnx

In [ ]:
def read_hdag(filename, num_entries):
    counter = 0
    data = defaultdict(list)
    with open(filename, 'r') as file:
        for line in file:
            if counter<num_entries and not line.startswith('%') and len(line.split())==2:
                counter+=1
                net_id, node_id = line.strip().split()
                data[net_id].append(node_id)
                
    if min(len(nodes) for nodes in data.values())<2:
        data = {net_id: nodes for net_id, nodes in data.items() if len(nodes)>1}
        print("There are singleton hyperedges which have been removed.")
    return data

In [ ]:
hdag = read_hdag(filename, num_entries)

---
# Show DAG

In [ ]:
G = nx.DiGraph()
G.add_edges_from([(net[0], net[i]) for net in hdag.values() for i in range(1,len(net))])

In [ ]:
pos = nx.drawing.nx_pydot.graphviz_layout(G, prog='dot') # this will also be used for hypergraphs later

In [ ]:
fig = plt.figure()
fig.set_size_inches(25, 10)
nx.draw_networkx_nodes(G, pos, node_color='lightblue', node_size=750)
nx.draw_networkx_edges(G, pos, edge_color='blue', arrowsize=30)
nx.draw_networkx_labels(G, pos, font_size=20)
plt.savefig("dag.jpg")
plt.show()

---
# Show HyperDAG (source nodes colored)

In [ ]:
nets = {int(index): tuple(net) for index, net in hdag.items()}
H = hnx.Hypergraph(nets)
sources = {next(iter(net.elements.items()))[0]: net.uid  for net in H.edges()}
# write new attribute source_of to nodes
for v in H.nodes():
    v.source_of = sources.get(v.uid)

In [ ]:
settings = {
    'width': 25,
    'height': 10,
    'xstretch': 1,
    'ystretch': 4,
    'net_padding': 0.2,
    'source_size': 0.8,
    'sink_size': 0.5,
}    

In [ ]:
cmap = plt.cm.viridis
colors = cmap(plt.Normalize(0, len(hdag))(range(len(hdag))))

fig = plt.figure()
fig.set_size_inches(settings['width'], settings['height'])

hnx.drawing.draw(H,
                 pos={key: (settings['xstretch']*x, settings['ystretch']*y) for key, (x,y) in pos.items()}, # use pos from digraph (cannot use graphviz.dot on hypergraphs)
                 edges_kwargs={
                     'edgecolors': colors,
                     'dr': settings['net_padding']
                  },
                 nodes_kwargs={
                    'facecolors': [colors[v.source_of] if v.source_of != None else 'gray' for v in H.nodes()]
                  },
                 node_radius={
                     v.uid: settings['source_size'] if v.uid in sources else settings['sink_size'] for v in H.nodes()
                  }
                )
fig.savefig("hyperdag.jpg")
plt.show()